In [ ]:
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, interact_manual
from ipywidgets import HTML, Layout, Dropdown, Output, Textarea, VBox, Label, Text, RadioButtons, Accordion

In [ ]:
#for asynchronous widgets
#didn't work as intended may come back to this solution
%gui asyncio
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [ ]:
header = HTML("<h1>Cryptocurrency Analyzer</h1>", layout=Layout(height='auto'))
header.style.text_align='center'
options = {('Find an account based on a person\'s username using Sherlock',1), 
         ('Find an account based on a person\'s email',2), 
         ('Analyze an account you already know',3)}

functionselector = RadioButtons( options = list(options),layout=Layout(width='auto'),value=None)
center = VBox([Label("Please select an option from the list below:"),functionselector], value=functionselector.value)
footer = Button(description="confirm", value=False)
start = AppLayout(center=center,
          header=header,
          left_sidebar=None,
          right_sidebar=None,
          footer= footer,
          grid_gap="10px")

In [ ]:
#address menu
#def addrmenu():
addrheader = HTML("<h1>Address Analyzer</h1>", layout=Layout(height='auto'))
addrheader.style.text_align='center'
coins = {('BITCOIN','BTC'), ('ETHEREUM','ETH'), ('LITECOIN','LTC'), ('DASH','DASH')}

currencyselector = Dropdown( options = list(coins),
                            layout=Layout(width='auto', positioning='left'),
                           value = 'BTC')

addrtext = widgets.Text(disabled=False,value='',
        layout=Layout(width='auto'))

excel = widgets.ToggleButton(
        value=False,
        description='Excel',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='check'
    )

transactioncount = widgets.BoundedIntText(
    value=15,
    min=0,
    max=10000,
    step=1,
    disabled=False,
    layout=Layout(width='auto'),
    )

button = widgets.Button(description="confirm", value=False)

menu = AppLayout(center=None,
          header=addrheader,
          left_sidebar=VBox([Label("Currency:"),
                             currencyselector,

                             Label("Address:"),
                             addrtext,

                             Label('Number of Trannsactions:'),
                             transactioncount,

                             Label('Export Options:'),
                             excel,
                            ]),
          right_sidebar=None,
          footer = button,
          height='auto',
          grid_gap="10px")

In [ ]:
def printlist(listin):
    vlist = []
    for i in listin:
        vlist.append(Label(i))
    return Vbox(vlist)

In [19]:
#address display

#menu container
addressmenu = AppLayout(center=None,
              header=None,
              left_sidebar=None,
              right_sidebar=None,
              footer =None)


def addrdisplay(currency, address, limit, excel):
    out.clear_output()
    addressinfo = collectdata(address, currency, limit, excel)
    infoout = printlist(addressinfo)
                        
    addrheader = HTML("<h1>Address Display</h1>", layout=Layout(height='auto'))
    addrheader.style.text_align='center'

    excel = widgets.ToggleButton(
        value=False,
        description='Excel',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='check'
    )

    addrtext = widgets.Text(disabled=False,
        layout=Layout(width='auto'))
    
    transactioncount = widgets.BoundedIntText(
        value=15,
        min=0,
        max=10000,
        step=1,
        disabled=False,
        layout=Layout(width='auto'),
    )
    
    addrmenu = AppLayout(center=None,
              header=addrheader,
              left_sidebar=infoout,
              right_sidebar=transaction(address, currency, limit),
              footer = VBox([Label(" "), confirm]),
              height='auto',
              grid_gap="10px")
    with out:
        display(addrmenu)
        
    addressmenu = addrmenu

In [ ]:
def collectdata(addr, coin, limit, export):
    from BlockCypher import overview
    overview = overview(addr, coin, 'USD')
    return overview

In [ ]:
def home(b):
    out.clear_output()
    functionselector.value
    return True

In [ ]:
#functions for building graph
def buildgraph(addr, addrlist):
    w = GraphWidget()
    
    nodelist = [{"id": 0, "properties": {"label": addr}}]
    edgelist = []
    i = 0
    for address in addrlist:
        i += 1
        nodelist.append({"id": i, "properties": {"label": address, "Transactions": addrlist[address]}})
        edgelist.append({"id": i, "start": 0, "end": i, "properties": {"label": "knows"}})
    
    w.nodes = nodelist
    w.edges = edgelist
    w.directed = True
    return w
    
def listtodict(addrlist):
    addrdict = {}
    for i in addrlist:
        if addrlist.count(i) > 1:
            addrdict[i] = addrlist.count(i)
    return addrdict
    
#call with
def transactions(address, coin, end):
    # still collect data for eth and btc from respective apis
    # able to collect more transactions in less api calls
    if coin == 'ETH' or coin == 'BTC':
        addrlist = currencyTX(address, end, coin)
    else:
        from BlockCypher import addrfull
        addrlist = addrfull(address, coin, 'USD', end)
        
        addrs = listtodict(addrlist)
        return buildgraph(address, addrs)


In [18]:
out = widgets.Output()
with out:        
    display(start)
i = footer.on_click(home)
out

Output()

In [17]:
out = widgets.Output()
with out:
    #display(menu)
    interact_manual(addrdisplay, currency=currencyselector.value, address=addrtext.value, limit=transactioncount.value, excel=excel.value);
    
out

Output()